# CSE 3683 Lab 1

The assignment for this notebook is to implement local search algorithms such as hill-climbing, random restart, and simulated annealing to perform function optimization.

In [4]:
# import required libraries
import time
import random
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [5]:
# Create a complex objective function that we need to maximize.
# Here, the objective function is simply defined as an equation but
# more generally, it could represent natural phenomena, physical laws, or mathematical models
#   X: a 2-dimensional floating-point vector consisting of an x-value and a y-value
#   returns: a scalar floating-point value representing the output of the objective function
def objective_function(X):
    x = X[0]
    y = X[1]
    value = 3 * (1 - x) ** 2 * math.exp(-x ** 2 - (y + 1) ** 2) - 10 * (x / 5 - x ** 3 - y ** 5) * math.exp(
        -x ** 2 - y ** 2) - (1 / 3) * math.exp(-(x + 1) ** 2 - y ** 2)
    return value

# to limit the search space for this problem, we will only consider solutions
# where x ranges from -4 to 4 and y ranges from -4 to 4
lower_bounds = [-4, -4]
upper_bounds = [4, 4]

In [6]:
x = -0.1 #putting in different numbers to make it higher
y = -0.1 #for both
print(objective_function([x,y]))

1.638096082415731


A 3D plot of the objective function can be viewed [here](https://c3d.libretexts.org/CalcPlot3D/index.html?type=z;z=3(1-x)%5E2*exp(-x%5E2-(y+1)%5E2)-10(x/5-x%5E3-y%5E5)*exp(-x%5E2-y%5E2)-(1/3)*exp(-(x+1)%5E2-y%5E2);visible=true;umin=-4;umax=4;vmin=-4;vmax=4;grid=30;format=normal;alpha=-1;hidemyedges=true;constcol=rgb(255,0,0);view=0;contourcolor=red;fixdomain=false;contourplot=true;showcontourplot=false;firstvalue=-1;stepsize=0.2;numlevels=11;list=;uselist=false;xnum=46;ynum=46;show2d=false;hidesurface=false;hidelabels=true;showprojections=false;surfacecontours=true;projectioncolor=rgba(255,0,0,1);showxygrid=false;showxygridonbox=false;showconstraint=false&type=window;hsrmode=3;nomidpts=true;anaglyph=-1;center=-5.2487653277286155,6.815843602039553,5.098503557610455,1;focus=0,0,0,1;up=0.39284920127083023,-0.3373981166615778,0.8554718089651412,1;transparent=false;alpha=140;twoviews=false;unlinkviews=false;axisextension=0.7;xaxislabel=x;yaxislabel=y;zaxislabel=z;edgeson=true;faceson=true;showbox=true;showaxes=true;showticks=true;perspective=true;centerxpercent=0.5;centerypercent=0.5;rotationsteps=30;autospin=true;xygrid=false;yzgrid=false;xzgrid=false;gridsonbox=true;gridplanes=false;gridcolor=rgb(128,128,128);xmin=-4;xmax=4;ymin=-4;ymax=4;zmin=-4;zmax=4;xscale=2;yscale=2;zscale=2;zcmin=-8;zcmax=8;xscalefactor=1;yscalefactor=1;zscalefactor=1;tracemode=0;keep2d=false;zoom=0.89)

In [7]:
# Let's try to randomly generate several random inputs to the objective function
# and manually observe how the outputs change with different inputs

X = np.random.uniform(lower_bounds, upper_bounds)
value = objective_function(X)
print('objective_function(%.3f, %.3f) = %.3f' % (X[0], X[1], value))

objective_function(-1.157, 0.983) = -0.451


# Part A: Hill-climbing

In [36]:
# TODO: Hill-climbing search algorithm: a loop that continually moves in the direction of increasing value.
# It terminates when it reaches a “peak” where no neighbor has a higher value.
#   objective function: function to be maximized
#   initial_state: initial (x, y) vector
#   step_size: numerical interval by which to change the current (x,y) state to generate a new neighboring state
#   print_iters: set to True to print out the current value at each iteration
#   returns: best [x, y] solution found
def hill_climbing(objective_function, initial_state = np.array([0, 0]), step_size = 0.01, print_iters=True):

  # set the starting point of the search algorithm
  current_state = initial_state
  current_value = objective_function(current_state)

  # loop until a peak is found
  i = 0
  while True:
    # Step 1: create a list of neighboring states to the current state
    neighbors = [
        current_state + np.array([step_size, 0]),
        current_state - np.array([step_size, 0]),
        current_state + np.array([0, step_size]),
        current_state - np.array([0, step_size])
    ]

    # Step 2: calculate the objective function at each of the neighboring states
    neighbor_values = [objective_function(n) for n in neighbors]

    # Step 3: determine the highest-valued neighboring state
    best_neighbor_idx = np.argmax(neighbor_values)
    best_neighbor = neighbors[best_neighbor_idx]
    best_value = neighbor_values[best_neighbor_idx]

    # EXTRA CRED 1: a)First-choice hill climbing -> randomly pick a neighbor until finding a better one
    #shuffle neighbors
    np.random.shuffle(neighbors)
    for neighbor in neighbors:
        neighbor_value = objective_function(neighbor)
        #accept first BETTER neigh
        if neighbor_value > current_value:
            best_neighbor = neighbor
            best_value = neighbor_value
            break


    # Step 4: compare the highest value among neighboring states to the current value
    #         if the latter is higher, we have found a peak -> return the current state
    #         if the former is higher, assign current state to be the best neighbor state

    if print_iters:
      print('iteration: {}, current_state: {}, current_value: {}'.format(i, current_state, current_value))
    i += 1

    if best_value <= current_value:
      break
    current_state = best_neighbor
    current_value = best_value
  return current_state


    # this break statement is added temporarily to prevent infinite loops
    # once the exit condition in Step 4 is implemented, the break statement can be removed



In [37]:
hill_climbing_solution = hill_climbing(objective_function)
print('Hill climbing solution is:', hill_climbing_solution)

iteration: 0, current_state: [0 0], current_value: 0.9810118431238463
iteration: 1, current_state: [ 0.   -0.01], current_value: 1.0032064894985684
iteration: 2, current_state: [-0.01 -0.01], current_value: 1.043242185894391
iteration: 3, current_state: [-0.01 -0.02], current_value: 1.0661201739582875
iteration: 4, current_state: [-0.01 -0.03], current_value: 1.0892346424891541
iteration: 5, current_state: [-0.01 -0.04], current_value: 1.1125801030489684
iteration: 6, current_state: [-0.01 -0.05], current_value: 1.136150470345207
iteration: 7, current_state: [-0.01 -0.06], current_value: 1.15993894532078
iteration: 8, current_state: [-0.02 -0.06], current_value: 1.2020837215577855
iteration: 9, current_state: [-0.02 -0.07], current_value: 1.226527867582368
iteration: 10, current_state: [-0.02 -0.08], current_value: 1.251174178494552
iteration: 11, current_state: [-0.03 -0.08], current_value: 1.2940121845111747
iteration: 12, current_state: [-0.04 -0.08], current_value: 1.33656644211595

# Part B: Random Restart

In [42]:
# TODO: Improvement to the Hill-climbing search algorithm using random restarts
#   objective function: function to be maximized
#   lower_bounds: minimum allowable values for the input vector to the objective function
#   upper_bounds: maximum allowable values for the input vector to the objective function
#   step_size: numerical interval by which to change the current (x,y) state to generate a new neighboring state
#   num_restarts: how many times to restart hill-climbing
#   returns: best [x, y] solution found
def random_restart_hill_climbing(objective_function, lower_bounds, upper_bounds, step_size=0.01, num_restarts=10):
    best_solution = None
    # initialize with the lowest possible value
    best_value = -np.inf

    for i in range(num_restarts):
        # generate a random starting point within bounds
        initial_state = np.random.uniform(lower_bounds, upper_bounds)

        # run hill climbing from this random initial state
        solution = hill_climbing(objective_function, initial_state, step_size, print_iters=False)
        solution_value = objective_function(solution)

        #print iterations
        print(f"restart: {i+1}, hill climbing solution: {solution}, value: {solution_value}")

        # update the best solution if this one is better
        if solution_value > best_value:
            best_solution = solution
            best_value = solution_value


    return best_solution

In [43]:
random_restart_solution = random_restart_hill_climbing(objective_function, lower_bounds, upper_bounds)
print('Random restart hill climbing solution:', random_restart_solution)

restart: 1, hill climbing solution: [ 1.28666921 -0.00421844], value: 3.592480784870229
restart: 2, hill climbing solution: [ 1.28276764 -0.00884888], value: 3.592364078136922
restart: 3, hill climbing solution: [-0.01327897  1.58252005], value: 8.106061939083094
restart: 4, hill climbing solution: [-0.00935266  1.57712678], value: 8.105921883148858
restart: 5, hill climbing solution: [ 1.28548213 -0.00476748], value: 3.5924895745587624
restart: 6, hill climbing solution: [-0.01040584  1.58146276], value: 8.10620374307351
restart: 7, hill climbing solution: [-0.00686061  1.58369298], value: 8.106079661615704
restart: 8, hill climbing solution: [ 1.28319059 -0.0065125 ], value: 3.5924303894967324
restart: 9, hill climbing solution: [-21.11968384  18.37736207], value: -1e-323
restart: 10, hill climbing solution: [-0.46523366 -0.63412968], value: 3.776209925326438
Random restart hill climbing solution: [-0.01040584  1.58146276]


# Part C: Simulated Annealing

In [40]:
# TODO: Simulated annealing algorithm
#   objective function: function to be maximized
#   lower_bounds: minimum allowable values for the input vector to the objective function
#   upper_bounds: maximum allowable values for the input vector to the objective function
#   returns: best [x, y] solution found

def simulated_annealing(objective_function, lower_bounds, upper_bounds, initial_temp=1000, cooling_rate=0.99, min_temp=1e-3, step_size=0.1):
    # initialize state randomly within bounds
    current_state = np.random.uniform(lower_bounds, upper_bounds)
    current_value = objective_function(current_state)

    best_state = np.copy(current_state)
    best_value = current_value
    temperature = initial_temp

    print_iter = 0
    while temperature > min_temp:

        print_iter += 1

        # adding small rando steps
        new_state = current_state + np.random.uniform(-step_size, step_size, size=len(lower_bounds))

        # make sure in bounds
        new_state = np.clip(new_state, lower_bounds, upper_bounds)
        new_value = objective_function(new_state)

        # compute change
        delta = new_value - current_value

        # accept new state if it's better or with probability based on temperature
        if delta > 0 or np.random.rand() < np.exp(delta / temperature):
            current_state, current_value = new_state, new_value

            # update best found solution
            if new_value > best_value:
                best_state, best_value = new_state, new_value

        #print iterations
        #commented out to see better
        #print(f"iteration {print_iter}: temperature = {temperature:.4f}, current value = {current_value:.4f}, best value = {best_value:.4f}")

        # decrease temperature
        temperature *= cooling_rate

    return best_state

In [41]:
simulated_annealing_solution = simulated_annealing(objective_function, lower_bounds, upper_bounds)
print('Simulated annealing solution is:', simulated_annealing_solution)

Simulated annealing solution is: [ 2.07684707 -3.97303918]
